In [25]:
import requests
import logging
import pandas as pd
import simfin as sf
from simfin.names import *

class SimFinAPIError(Exception):
    """Custom exception for SimFin API errors."""
    pass

class PySimFin:
    def __init__(self, api_key: str):
        """Initialize the API wrapper with the given API key."""
        self.api_key = api_key
        sf.set_api_key(self.api_key)
        sf.set_data_dir('~/simfin_data/')
        
        # Configure logging
        logging.basicConfig(
            filename='simfin_api.log', 
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        logging.info("SimFin API Wrapper initialized.")
    
    def get_share_prices(self, ticker: str, start: str, end: str) -> pd.DataFrame:
        """Retrieve historical stock price data for the given ticker and date range."""
        logging.info(f"Fetching historical share prices for {ticker} from {start} to {end}.")
        try:
            prices = sf.load_shareprices(variant='daily', market='us')  # Load daily historical data
            available_tickers = prices.index.get_level_values(0).unique()
            
            if ticker not in available_tickers and f"{ticker}_US" in available_tickers:
                ticker = f"{ticker}_US"
            
            if ticker not in available_tickers:
                raise SimFinAPIError(f"Ticker {ticker} not found in historical share prices dataset.")
            
            prices = prices.loc[ticker]
            prices.reset_index(inplace=True)
            
            print("Earliest available date:", prices['Date'].min(), "Latest available date:", prices['Date'].max())
            prices_filtered = prices[(prices['Date'] >= start) & (prices['Date'] <= end)]
            
            if prices_filtered.empty:
                logging.warning(f"No historical share price data available for {ticker} in the given period.")
                return pd.DataFrame({"Error": ["No data available for selected dates."]})
            return prices_filtered
        except Exception as e:
            logging.error(f"Error fetching historical share prices: {e}")
            raise SimFinAPIError(f"Error fetching historical share prices: {e}")
    
    def get_financial_statement(self, ticker: str, start: str, end: str) -> pd.DataFrame:
        """Retrieve financial statement data for the given ticker and date range."""
        logging.info(f"Fetching financial statements for {ticker} from {start} to {end}.")
        try:
            financials = sf.load_income(variant='annual', market='us')
            if ticker not in financials.index:
                raise SimFinAPIError(f"Ticker {ticker} not found in financial statements dataset.")
            
            financials = financials.loc[ticker]
            financials.reset_index(inplace=True)
            financials_filtered = financials[(financials['Fiscal Year'] >= int(start[:4])) & (financials['Fiscal Year'] <= int(end[:4]))]
            
            if financials_filtered.empty:
                logging.warning(f"No financial data available for {ticker} in the given period.")
            return financials_filtered
        except Exception as e:
            logging.error(f"Error fetching financial statements: {e}")
            raise SimFinAPIError(f"Error fetching financial statements: {e}")

# Example Usage (Replace 'your_api_key' with an actual key)
if __name__ == "__main__":
    api_key = "e1c75cc5-3bca-4b0c-b847-6447bd4ed901"  # Replace with your actual SimFin API key
    simfin = PySimFin(api_key)
    
    # Fetch share prices
    prices_df = simfin.get_share_prices("AAPL", "2023-01-01", "2024-03-01")
    print(prices_df.head())
    
    # Fetch financial statements
    financials_df = simfin.get_financial_statement("AAPL", "2023-01-01", "2024-03-01")
    print(financials_df.head())


Dataset "us-shareprices-daily" not on disk.
- Downloading ... 100.0%
- Extracting zip-file ... Done!
- Loading from disk ... 

C:\Users\Samir\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,


Done!
Earliest available date: 2019-04-09 00:00:00 Latest available date: 2024-03-12 00:00:00
          Date  SimFinId    Open    High     Low   Close  Adj. Close  \
941 2023-01-03    111052  130.28  130.90  124.17  125.07      123.64   
942 2023-01-04    111052  126.89  128.66  125.08  126.36      124.91   
943 2023-01-05    111052  127.13  127.77  124.76  125.02      123.59   
944 2023-01-06    111052  126.01  130.29  124.89  129.62      128.13   
945 2023-01-09    111052  130.47  133.41  129.89  130.15      128.66   

        Volume  Dividend  Shares Outstanding  
941  112117471       NaN        1.589272e+10  
942   89113633       NaN        1.589272e+10  
943   80962708       NaN        1.589272e+10  
944   87754715       NaN        1.589272e+10  
945   70790813       NaN        1.589272e+10  
Dataset "us-income-annual" on disk (0 days old).
- Loading from disk ... Done!
  Report Date  SimFinId Currency  Fiscal Year Fiscal Period Publish Date  \
4  2023-09-30    111052      USD    

C:\Users\Samir\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\simfin\load.py:154: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(path, sep=';', header=0,
